In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import re
import json
import networkx as nx
from netgraph import Graph
from datetime import datetime
# from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from category_encoders import one_hot
import torch
# import cudf
# from cuml.manifold.umap import UMAP
# from cuml.decomposition import PCA as cumlPCA
from positional_encodings.torch_encodings import PositionalEncoding1D
import matplotlib.pyplot as plt
from lib.utilities import Repository



In [2]:
repo = Repository('./session_repositories/actions.tsv','./session_repositories/displays.tsv','./raw_datasets/')

In [3]:
repo.actions[repo.actions['action_type'] == 'filter']['action_params']

5       {'field': 'info_line', 'term': 'reply', 'condi...
6       {'field': 'highest_layer', 'term': 'ICMP', 'co...
7       {'field': 'highest_layer', 'term': 'SMB', 'con...
8       {'field': 'tcp_dstport', 'term': '443', 'condi...
9       {'field': 'tcp_dstport', 'term': '445', 'condi...
                              ...                        
2407    {'condition': 16, 'term': 'SYN, ACK', 'field':...
2412    {'condition': 16, 'term': 'SYN, ACK', 'field':...
2416    {'term': '1957', 'field': 'tcp_srcport', 'cond...
2452    {'field': 'highest_layer', 'term': 'TCP', 'con...
2453    {'field': 'highest_layer', 'term': 'SMB', 'con...
Name: action_params, Length: 546, dtype: object

In [4]:
og_columns = ['captured_length', 'length', 'tcp_stream', 'number', 'eth_dst', 'eth_src', 
                'highest_layer', 'info_line', 'interface_captured', 'ip_dst', 'ip_src', 
                'sniff_timestamp', 'tcp_dstport', 'tcp_srcport']

In [5]:
og_column_types = {
                        'captured_length':'ratio', 
                        'length':'ratio', 
                        'tcp_stream':'ratio', 
                        'number':'ordinal', 
                        'eth_dst':'nominal', 
                        'eth_src':'nominal', 
                        'highest_layer':'ordinal', 
                        'info_line':'textual', 
                        'interface_captured':'ordinal', 
                        'ip_dst':'nominal', 
                        'ip_src':'nominal', 
                        'sniff_timestamp':'time', 
                        'tcp_dstport':'nominal', 
                        'tcp_srcport':'nominal', 
                        '~!~':'misc'
                    }

In [6]:
og_col_index = {}
for i in range(len(og_columns)):
    og_col_index[og_columns[i]] = i

In [7]:
def connected_components(edges):
    edge_list = []
    edge_attr = {}
    for edge in edges:
        edge_list.append((edge[0], edge[1]))
        edge_attr[(edge[0], edge[1])] = edge[2]
    G = nx.from_edgelist(edge_list)
    nx.set_edge_attributes(G, edge_attr)

    cc = [[c, [e for e in G.subgraph(c).edges.data()], ] for c in nx.connected_components(G)]

    return cc

In [8]:
def replay_graph(graph):
    replays = []
    if graph.num_nodes > 2:
        for node in range(graph.num_nodes - 2):
            current_nodes = list(range(node + 2))
            replays.append(graph.subgraph(torch.tensor(current_nodes)))

    return replays

In [9]:
offset = {
    'project':0 * len(og_columns),
    'filter':1 * len(og_columns),
    'group':2 * len(og_columns),
    'count':3 * len(og_columns),
    'min':4 * len(og_columns),
    'max':5 * len(og_columns),
    'sum':6 * len(og_columns),
    'avg':7 * len(og_columns),
    'asc':8 * len(og_columns),
    'desc':9 * len(og_columns)
}

In [19]:
## Old
feats = {}
feats['root'] = np.zeros(10 * len(og_columns))
# for i in range(len(og_columns)):
#     feats['root'][i] = 1

for i, row in repo.actions.iterrows():
    raw_feat = np.zeros(10 * len(og_columns))
    u = int(row['parent_display_id'])
    v = int(row['child_display_id'])
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue

    if not bool(row['action_params']):
        action_params['field'] = 'number'

    if action_type == 'sort':
        direction = 'asc'
        if 'direction' in action_params:
            direction = action_params['direction']

        if direction == 'asc':
            raw_feat[offset['asc'] + og_col_index[action_params['field']]] = 1
        elif direction == 'desc':
            raw_feat[offset['desc'] + og_col_index[action_params['field']]] = 1
    elif action_type == 'project':
        if action_params['visible'] == 'false':
            raw_feat[offset['project'] + og_col_index[action_params['field']]] = 1
    elif action_type == 'group':
        raw_feat[offset['group'] + og_col_index[action_params['field']]] = 1
        aggs = action_params['aggregations']
        if len(aggs) > 0:
            for agg in aggs:
                raw_feat[offset[agg['type']] + og_col_index[agg['field']]] = 1
    elif action_type == 'filter':
        raw_feat[offset['filter'] + og_col_index[action_params['field']]] = 1
    else:
        raise NotImplementedError

    feats[v] = raw_feat.copy()

for i, row in repo.actions.iterrows():
    u = int(row['parent_display_id'])
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue
    
    raw_feat = np.zeros(10 * len(og_columns))
    # raw_feat[0] = 1
    if not u in feats:
        feats[u] = raw_feat.copy()

In [10]:
## Old
with open(f'./edge/esp_feats.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
## Old
feats = {}
feats['root'] = np.zeros(5)
# for i in range(len(og_columns)):
#     feats['root'][i] = 1

action_index = {'project':0, 'filter':1, 'group':2, 'sort':3}

for i, row in repo.actions.iterrows():
    raw_feat = np.zeros(4)
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue

    try:
        raw_feat[action_index[action_type]] = 1
    except:
        raise NotImplementedError

    feats[int(row['action_id'])] = raw_feat.copy()

with open(f'./edge/main_action.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
## Action Feats

feats = {}

# action_index = {'offset':0, 'project':1, 'filter':2, 'group':3, 'sort':4}
action_index = {'project':0, 'filter':1, 'group':2, 'sort':3}
# action_index = {'project':0, 'filter':1, 'group':2, 'aggregate':3, 'sort':4}

for i, row in repo.actions.iterrows():
    raw_feat = np.zeros(len(action_index.keys()))
    u = int(row['parent_display_id'])
    v = int(row['child_display_id'])
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue

    # if action_type == 'group':
    #     if len(action_params['aggregations']) > 0:
    #         action_type = 'aggregate'

    try:
        raw_feat[action_index[action_type]] = 1
    except:
        raise NotImplementedError

    feats[v] = raw_feat.copy()

for i, row in repo.actions.iterrows():
    u = int(row['parent_display_id'])
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue
    
    raw_feat = np.zeros(len(action_index.keys()))
    # raw_feat[0] = 1
    if not u in feats:
        feats[u] = raw_feat.copy()

with open(f'./edge/main_feats.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
## Five Action Feats

feats = {}

# action_index = {'offset':0, 'project':1, 'filter':2, 'group':3, 'sort':4}
action_index = {'project':1, 'filter':2, 'group':3, 'sort':4}
# action_index = {'project':0, 'filter':1, 'group':2, 'aggregate':3, 'sort':4}

for i, row in repo.actions.iterrows():
    raw_feat = np.zeros(len(action_index.keys()) + 1)
    u = int(row['parent_display_id'])
    v = int(row['child_display_id'])
    aid = int(row['action_id'])
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue

    # if action_type == 'group':
    #     if len(action_params['aggregations']) > 0:
    #         action_type = 'aggregate'

    try:
        raw_feat[action_index[action_type]] = 1
    except:
        raise NotImplementedError

    feats[aid] = raw_feat.copy()

# for i, row in repo.actions.iterrows():
#     u = int(row['parent_display_id'])
#     action_params = row['action_params']

#     if row['action_type'] == 'sort' and (not bool(row['action_params'])):
#         check_col = 'number'
#     else:
#         check_col = row['action_params']['field']

#     if not check_col in og_columns:
#         continue
    
#     raw_feat = np.zeros(len(action_index.keys()) + 1)
#     raw_feat[0] = 1
#     if not u in feats:
#         feats[u] = raw_feat.copy()

with open(f'./edge/act_five_feats.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
## Column Feats

# feats = {}

action_index = {}
for i in range(len(og_columns)):
    action_index[og_columns[i]] = i + 1

for i, row in repo.actions.iterrows():
    raw_feat = np.zeros(len(og_columns) + 1)
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue

    try:
        raw_feat[action_index[check_col]] = 1
    except:
        raise NotImplementedError

    feats[int(row['action_id'])] = raw_feat.copy()

with open(f'./edge/col_action.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

feats = {}
col_type_index = {}
keys = list(og_column_types.keys())
for i in range(len(keys)):
    col_type_index[keys[i]] = i

for i, row in repo.actions.iterrows():
    raw_feat = np.zeros(len(og_columns) + 1)
    u = int(row['parent_display_id'])
    v = int(row['child_display_id'])
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue

    try:
        raw_feat[action_index[check_col]] = 1
    except:
        raise NotImplementedError

    feats[v] = raw_feat.copy()

for i, row in repo.actions.iterrows():
    u = int(row['parent_display_id'])
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue
    
    raw_feat = np.zeros(len(og_columns) + 1)
    raw_feat[0] = 1
    if not u in feats:
        feats[u] = raw_feat.copy()

with open(f'./edge/col_feats.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

{'field': 'eth_src', 'aggregations': [], 'groupPriority': 0}
{'field': 'ip_src', 'aggregations': [], 'groupPriority': 1}
{'field': 'eth_src', 'aggregations': [{'field': 'length', 'type': 'avg'}], 'groupPriority': 0}
{'field': 'ip_src', 'aggregations': [{'field': 'length', 'type': 'avg'}], 'groupPriority': 0}
{'field': 'ip_src', 'aggregations': [], 'groupPriority': 0}
{'field': 'info_line', 'term': 'reply', 'condition': 16}
{'field': 'highest_layer', 'term': 'ICMP', 'condition': 512}
{'field': 'highest_layer', 'term': 'SMB', 'condition': 512}
{'field': 'tcp_dstport', 'term': '443', 'condition': 8}
{'field': 'tcp_dstport', 'term': '445', 'condition': 8}
{'field': 'ip_dst', 'term': '192.168.1.122', 'condition': 8}
{'field': 'ip_src', 'aggregations': [{'field': 'length', 'type': 'avg'}], 'groupPriority': 0}
{'field': 'ip_src', 'aggregations': [{'field': 'length', 'type': 'sum'}], 'groupPriority': 0}
{'field': 'info_line', 'term': 'Ech', 'condition': 16}
{'field': 'ip_dst', 'aggregations': 

In [17]:
 ## Condition Feats

op_index = {
            2: 1,
            4: 2,
            8: 3,
            16: 4,
            32: 5,
            64: 6,
            128: 7,
            256: 8,
            512: 9
        }

feats = {}

for i, row in repo.actions.iterrows():
    raw_feat = np.zeros(len(op_index) + 1)
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue

    cond = 0
    if 'condition' in action_params:
        cond = op_index[action_params['condition']]

    try:
        raw_feat[cond] = 1
    except:
        raise NotImplementedError

    feats[int(row['action_id'])] = raw_feat.copy()

with open(f'./edge/cond_action.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
## Type Feats

feats = {}
col_type_index = {}
values = list(set(og_column_types.values()))
for i in range(len(values)):
    col_type_index[values[i]] = i

for i, row in repo.actions.iterrows():
    raw_feat = np.zeros(len(col_type_index))
    u = int(row['parent_display_id'])
    v = int(row['child_display_id'])
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue

    try:
        raw_feat[col_type_index[og_column_types[check_col]]] = 1
    except:
        raise NotImplementedError

    feats[v] = raw_feat.copy()

for i, row in repo.actions.iterrows():
    u = int(row['parent_display_id'])
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue
    
    raw_feat = np.zeros(len(col_type_index))
    # raw_feat[0] = 1
    if not u in feats:
        feats[u] = raw_feat.copy()

with open(f'./edge/type_feats.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
## PCA Flattened

width = 8
feats = {}

for i, row in repo.actions.iterrows():
    raw_feat = np.zeros(len(og_columns) + 1)
    u = int(row['parent_display_id'])
    v = int(row['child_display_id'])
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue

    try:
        display_grids = None
        if os.path.isfile(f'./pca/displays/{width}/{v}.pickle'):
            with open(f'./pca/displays/{width}/{v}.pickle', 'rb') as fin:
                display_grids = pickle.load(fin)
        else:
            continue
        
        raw_feat = np.array([])
        for grid in display_grids:
            raw_feat = np.append(raw_feat, grid.flatten())
    except:
        raise NotImplementedError

    feats[v] = raw_feat.copy()

for i, row in repo.actions.iterrows():
    u = int(row['parent_display_id'])
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue
    
    display_grids = None
    raw_feat = np.array([])
    if os.path.isfile(f'./pca/displays/{width}/{u}.pickle'):
        with open(f'./pca/displays/{width}/{u}.pickle', 'rb') as fin:
            display_grids = pickle.load(fin)
            for grid in display_grids:
                raw_feat = np.append(raw_feat, grid.flatten())
    else:
        for key in feats:
            raw_feat = np.zeros(feats[key].shape[0])
            break
    
    if not u in feats:
        feats[u] = raw_feat.copy()

with open(f'./edge/pca_{width}_feats.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
width = 16
feats = {}

for i, row in repo.actions.iterrows():
    u = int(row['parent_display_id'])
    v = int(row['child_display_id'])
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue

    try:
        display_grids = None
        if os.path.isfile(f'./pca/displays/{width}/{v}.pickle'):
            with open(f'./pca/displays/{width}/{v}.pickle', 'rb') as fin:
                display_grids = pickle.load(fin)
        else:
            continue
    except:
        raise NotImplementedError

    feats[v] =  display_grids.copy()

for i, row in repo.actions.iterrows():
    u = int(row['parent_display_id'])
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue
    
    display_grids = None
    if os.path.isfile(f'./pca/displays/{width}/{u}.pickle'):
        with open(f'./pca/displays/{width}/{u}.pickle', 'rb') as fin:
            display_grids = pickle.load(fin)
    else:
        for key in feats:
            display_grids = np.zeros(feats[key].shape)
            break
    
    if not u in feats:
        feats[u] = display_grids.copy()

with open(f'./edge/pca_{width}_grids.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
## UMAP Flattened

method = 'umap'       
width = 16
feats = {}

for i, row in repo.actions.iterrows():
    raw_feat = np.zeros(len(og_columns) + 1)
    u = int(row['parent_display_id'])
    v = int(row['child_display_id'])
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue

    try:
        display_grids = None
        if os.path.isfile(f'./{method}/displays/{width}/{v}.pickle'):
            with open(f'./{method}/displays/{width}/{v}.pickle', 'rb') as fin:
                display_grids = pickle.load(fin)
        else:
            continue
        
        raw_feat = np.array([])
        for grid in display_grids:
            raw_feat = np.append(raw_feat, grid.flatten())
    except:
        raise NotImplementedError

    feats[v] = raw_feat.copy()

for i, row in repo.actions.iterrows():
    u = int(row['parent_display_id'])
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue
    
    display_grids = None
    raw_feat = np.array([])
    if os.path.isfile(f'./{method}/displays/{width}/{u}.pickle'):
        with open(f'./{method}/displays/{width}/{u}.pickle', 'rb') as fin:
            display_grids = pickle.load(fin)
            for grid in display_grids:
                raw_feat = np.append(raw_feat, grid.flatten())
    else:
        for key in feats:
            raw_feat = np.zeros(feats[key].shape[0])
            break
    
    if not u in feats:
        feats[u] = raw_feat.copy()

with open(f'./edge/{method}_{width}_feats.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
## Old

def not_related(col):
    if not col in og_columns:
        return True
    else:
        return False

In [ ]:
## Old

count = 0
for i, row in repo.actions[repo.actions['action_type'] == 'group'].iterrows():
    aggs = row['action_params']['aggregations']
    if len(aggs) > 0:
        count += 1
print(count)

In [13]:
## Old

col_feats = {}
agg_feats = {}
agg_offset = {'min':0, 'max':1, 'count':2, 'sum':3, 'avg':4}

for i, row in repo.actions.iterrows():
    u = int(row['parent_display_id'])
    v = int(row['child_display_id'])
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue
    
    col_feats[v] = []
    agg_feats[v] = []
    if action_type == 'group':
        try:
            aggs = action_params['aggregations']
            for agg in aggs:
                col_raw_feat = np.zeros(len(og_columns))
                agg_raw_feat = np.zeros(5)
                col_raw_feat[og_col_index[agg['field']]] = 1
                agg_raw_feat[agg_offset[agg['type']]] = 1
                col_feats[v].append(col_raw_feat)
                agg_feats[v].append(agg_raw_feat)
        except:
            raise NotImplementedError

for i, row in repo.actions.iterrows():
    u = int(row['parent_display_id'])
    v = int(row['child_display_id'])
    action_type = row['action_type']
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue
    
    if not u in col_feats:
        col_feats[u] = []
    
    if not u in agg_feats:
        agg_feats[u] = []

with open(f'./edge/agg_col_feats.pickle', 'wb') as fout:
    pickle.dump(col_feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'./edge/agg_type_feats.pickle', 'wb') as fout:
    pickle.dump(agg_feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
## Old

feats = {}
# feat_indices = {}
# feats['root'] = np.zeros(10 * len(og_columns))
# for i in range(len(og_columns)):
#     feats['root'][i] = 1

# index = 0
for i, row in repo.displays.iterrows():
    raw_feat = np.zeros(10 * len(og_columns))

    try:
        projection = json.loads(row['projected_fields'])['list']
    except:
        projection = []
        print(row['projected_fields'])

    try:
        filtering = json.loads(row['filtering'])['list']
    except:
        filtering = []
        print(row['filtering'])

    try:
        sorting = json.loads(row['sorting'])['list']
    except:
        sorting = []
        print(row['sorting'])

    try:
        grouping = json.loads(row['grouping'])['list']
    except:
        grouping = []
        print(row['grouping'])

    try:
        aggregations = json.loads(row['aggregations'])['list']
    except:
        aggregations = []
        print(row['aggregations'])

    unrelated = False
    for item in projection:
        if not_related(item['field']):
            unrelated = True
            break

        raw_feat[offset['project'] + og_col_index[item['field']]] = 1

    for item in filtering:
        if not_related(item['field']):
            unrelated = True
            break

        raw_feat[offset['filter'] + og_col_index[item['field']]] = 1

    for item in sorting:
        if not_related(item['field']):
            unrelated = True
            break

        direction = 'asc'
        if 'direction' in item:
            direction = item['direction']

        if direction == 'asc':
            raw_feat[offset['asc'] + og_col_index[item['field']]] = 1
        elif direction == 'desc':
            raw_feat[offset['desc'] + og_col_index[item['field']]] = 1

    for item in grouping:
        if not_related(item['field']):
            unrelated = True
            break

        raw_feat[offset['group'] + og_col_index[item['field']]] = 1

    for item in aggregations:
        if not_related(item['field']):
            unrelated = True
            break

        raw_feat[offset[item['type']] + og_col_index[item['field']]] = 1

    if unrelated:
        continue
    
    # feat_indices[int(row['display_id'])] = index
    # feats.append(raw_feat.copy())
    # index += 1
    feats[int(row['display_id'])] = raw_feat.copy()

for i, row in repo.actions.iterrows():
    u = int(row['parent_display_id'])
    action_params = row['action_params']

    if row['action_type'] == 'sort' and (not bool(row['action_params'])):
        check_col = 'number'
    else:
        check_col = row['action_params']['field']

    if not check_col in og_columns:
        continue

    if not u in feats:
        feats[u] = np.zeros(10 * len(og_columns))

with open(f'./edge/mix_feats.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
## Old

with open(f'./edge/feats.pickle', 'wb') as fout:
    pickle.dump(feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'./edge/feat_indices.pickle', 'wb') as fout:
    pickle.dump(feat_indices, fout, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
## Old

cudf_centered = cudf.DataFrame(feats - feats.mean())
mapper_pca_all = cumlPCA(n_components = feats.shape[1], whiten=True, output_type='numpy').fit(cudf_centered)

exp_var_all = mapper_pca_all.explained_variance_ratio_

total_exp_var_all = 0.0
pc_count = 0
for var in exp_var_all:
    total_exp_var_all += var
    pc_count += 1
    if total_exp_var_all >= 0.8 and pc_count % 2 == 0:
        break

print(f'Total explained variance ratio={total_exp_var_all}, Principal Components={pc_count}')

In [24]:
## Old

mapper_pca_all = cumlPCA(n_components = 16, whiten=True, output_type='numpy').fit(cudf_centered)
pca_feats = mapper_pca_all.transform(cudf_centered)

In [26]:
## Old

with open(f'./edge/feats2.pickle', 'wb') as fout:
    pickle.dump(pca_feats, fout, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'./edge/feat_indices.pickle', 'wb') as fout:
    pickle.dump(feat_indices, fout, protocol=pickle.HIGHEST_PROTOCOL)